In [10]:
import os
import warnings
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras import layers, models
from keras.datasets import mnist
from keras.utils import to_categorical

warnings.filterwarnings('ignore')

In [11]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))

### Conv2D
model.add(layers.Conv2D(32,(3,3), padding='valid', activation='relu', input_shape=(28,28,1)))

    첫 번째 인자: 컨볼루션 필터의 수

    두 번째 인자, 컨볼루션 커널의 (행,열)

    padding: 경계 처리 방법을 정의

        'valid'(padding 이미지 위쪽):유효한 영역만 출력, 따라서 출력 이미지 사이즈는 입력 사이즈보다 적음

                4x4 -> 3x3 -> 2x2 

        'same'(padding 이미지 아래쪽):출력 이미지 사이즈가 입력 이미지 사이즈와 동일
               
               가장자리에 가로,세로로 1줄씩 추가 한 후 Conv2D를 돌려서 사이즈가 줄어들지 않게 함

               예) 3x3이 있었다고 한다면, 가장자리에 1줄씩 추가해서 4x4로 시작, 결과값은 3x3이 되지만, 한쪽 방향으로 쏠리게 되어있음
    
    input_shape: 샘플 수를 제외한 입력 형태를 정의. 모델에서 첫 레이어일 때만 정의하면 됨

        (행, 열, 채널 수)로 정의, 흑백영상인 경우에는 채널이 1이고, 컬러 영상인 경우에는 채널이 3으로 설정

    activation: 활성화 함수 설정

        'linear': 디폴트 값, 입력뉴런과 가중치로 계산된 결과값이 그대로 출력

        'relu': rectifier 함수, 은닉층에 주로 쓰임

        'sigmoid': 시그모이드 함수, 이진 분류 문제에서 출력층에 주로 쓰임

        'softmax': 소프트맥스 함수, 다중 클래스 분류 문제에서 출력층에 주로 쓰임    
    
    Con2D => 이미지를 나눠서(예를들면 이미지를 8등분해서) 그 특징들을 배움

    이미지를 배우려면 3차원 데이터가 필요함( 가로+세로, 컬러 혹은 흑백, 강조)

    예를들면 흑백은 [28,28,1], 컬러는 [28,28,[rgb값]]

### Max Pooling 레이어

max_pooling 이미지 참고!

    그림에서 외각선일 경우(이미지의 그라데이션 차이가 많이 날 경우) 수치가 높아지는데

    그럴 경우 다른 바탕색은 제거하고 중요한 외각선 데이터만 뽑아내는것


In [12]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
Total params: 55,744
Trainable params: 55,744
Non-trai

In [13]:
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

### Flatten()

Conv2D나 Maxpooling으로 나온 결과값 (2D~3D) 데이터를 1D 타입으로 길게 만들어 주는 것


In [14]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 576)              

In [16]:
# 데이터를 다운 및 변형
(train_images, train_labels),(test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000,28,28,1)) # 60000개를 28x28 그리고 흑백이니까 1로 바꿔달라
train_images = train_images.astype('float32') # 실수형으로 데이터 변형

# 테스트 데이터도 변경
test_images = test_images.reshape((10000,28,28,1))
test_images = test_images.astype('float32')

# 라벨 데이터 변경
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [17]:
# 모델 컴파일
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
# 모델 학습
model.fit(train_images, train_labels, epochs=7, batch_size=64)

Epoch 1/7
938/938 [==============================] - 15s 15ms/step - loss: 0.3093 - accuracy: 0.9337
Epoch 2/7
938/938 [==============================] - 15s 16ms/step - loss: 0.0644 - accuracy: 0.9819
Epoch 3/7
938/938 [==============================] - 15s 15ms/step - loss: 0.0503 - accuracy: 0.9866
Epoch 4/7
938/938 [==============================] - 15s 16ms/step - loss: 0.0452 - accuracy: 0.9886
Epoch 5/7
938/938 [==============================] - 15s 16ms/step - loss: 0.0428 - accuracy: 0.9891
Epoch 6/7
938/938 [==============================] - 15s 16ms/step - loss: 0.0416 - accuracy: 0.9899
Epoch 7/7
938/938 [==============================] - 15s 16ms/step - loss: 0.0404 - accuracy: 0.9903


In [20]:
# 데이터 테스트
test_loss, test_acc = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.1112 - accuracy: 0.9834


CNN(Convolutional Neural Network)

    CNN은 이미지를 인식하기위해 패턴을 찾는데 특히 유용함

    데이터에서 직접 학습하고 패턴을 사용해 이미지를 분류(특징을 수동으로 추출 할 필요X)

    이러한 장점때문에 자율주행자동차, 얼굴인식과 같은 객체인식이나 computer vision이 필요한 분야에 많이 사용되는 중

CNN을 사용하기 위해서는

    Conovolution(합성곱) 사용!

        사진을 쪼개서 확대한 후 가르침